Renderer Name:
It is usually a file extension, eg. vxml, ccxml

In [ ]:
vxml

MIME type:
Provide the MIME type for given file extension

In [ ]:
application/voicexml+xml

Underscore JS template:

In [ ]:
<?xml version="1.0" encoding="UTF-8"?>
<vxml version="2.1">

  ## Note: All over-rides below must be over-ridden in your flow, preferably your entry flow

  ## Over-ride for a different timeout between user prompts
  #if (! $timeout) #set($timeout = 10) #end

  ## Mandatorily over-ride this with the current caller's locale based on how your application stores it
	#if (! $locale) #set($locale = "English") #end

  ## over-ride this if you want to use a different skip character
  #if (! $skipChars) #set($skipChars = "*") #end
  
  #set($ext = ".mp3")

  ## Mandatorily over-ride this with the new flow name that you want to skip to if the skipChars is pressed
  #if ($internal.callDirection == "OUTGOING" && $skipFlow && $skipFlow != "")
	<link event="SkipEvent">
		<grammar xml:lang="en-us" version="1.0" root="skipRule" mode="dtmf">
			<rule id="skipRule" scope="public">
				<item>${skipChars}</item>
			</rule>
		</grammar>
	</link>

	<catch event="SkipEvent">
		<submit next="${internal.nextURL}?jumpTo=${skipFlow}" />
	</catch>
  #end

<%

  // Toggle this, if you want to observe renderer output in your browser console
  var DEBUG = true;

	var sayAsTTS = function(txt, element) {
		var out = '',
				bargeIn = element.bargeIn ? true : false;

		if (txt.match(/^\s*$/)) {
			return out;
		}
    if (DEBUG) {
      console.log('[TTS]: ' + txt);
    }
		out += '\n\t\t<prompt bargein="';
		if (element.type === 'field') {
      out += bargeIn;
		  out += '" timeout="';
      out += '${timeout}';
      out += 's';
		} else {
      out += true;
    	out += '" bargeintype="hotword';
    }
		out += '">';
		out += txt;
		out += '</prompt>';
		return out;
	}

	var sayAsAudio = function(txt, mappings, element) {
		var out = '',
				bargeIn = element.bargeIn ? true : false;
		if (txt.match(/^\s*$/)) {
			return out;
		}
		out += '\n\t\t<prompt bargein="';
		out += bargeIn;
		if (element.type === 'field') {
		  out += '" timeout="';
		  out += '${timeout}';
		  out += 's';
		}
		out += '">';
		if (txt.indexOf('$') === -1) {
			// Not dynamic
			out += '\n\t\t\t<audio src="https://idi-audio.s3.us-west-2.amazonaws.com/c-care/$locale/' + mappings[0].mapping + '$ext' +'" />';
		} else {
      
      out += '\n\t\t\t<audio src="https://idi-audio.s3.us-west-2.amazonaws.com/c-care/$locale/' + txt + '$ext';
			//out += '\n\t\t\t<audio src="http://cflinstalltest.cloudapp.net/audio/openmrs/en/' + txt + '.mp3" />';
		}
    out += '" />';
		out += '\n\t\t</prompt>';
		return out;
	}

	var speak = function(element, txt, audio) {
		var out = '',
			bq = "`",
				phrase = '',
				audioStream = false,
				numberText = '<prosody rate="-20%"><say-as interpret-as="vxml:digits">$1</say-as></prosody>',
				dtText = '<prosody rate="-20%"><say-as interpret-as="vxml:date">$DateUtil.dateToString($1, "yyyyMMdd")</say-as></prosody>',
				numberServerText = "#set($chars = $1.toString().toCharArray())\n#foreach ($char in $chars)\n\t `num_00${char}` \n#end",
				percentageText = "#set($noValue = $1)\n #if($noValue < 10) `num_00${noValue}` #elseif ($noValue < 100) `num_0${noValue}` #else `num_${noValue}` #end",
				dtServerText = '',
				i, j;

		if (! audio) {
			audio = {};
		}
		if (DEBUG) {
      console.log('speak: ' + txt);
    }

		// Place custom delimiters here
		txt = txt.replace(/__\s*(.*?)\s*__/g, numberText);
		txt = txt.replace(/_\[\s*(.*?)\s*\]_/g, numberServerText);
		txt = txt.replace(/_%\s*(.*?)\s*%_/g, percentageText);
		txt = txt.replace(/%%\s*(.*?)\s*%%/g, dtText);
		txt = txt.replace(/%\[\s*(.*?)\s*\]%/g, dtServerText);

		if (DEBUG) {
      console.log('speak (transformed): ' + txt);
    }

		for (i = 0; i < txt.length; i+=1) {
			if (txt[i] !== '`') {
				phrase += txt[i];
			} else {
				if (! audioStream) {
					// backquote opened, start a audio stream
					audioStream = true;
					if (phrase) {
						out += sayAsTTS(phrase, element);
					}
					// reset phrase
					phrase = '';
				} else {
					// closed audio stream
          if (DEBUG) {
						console.log('[AUDIO]: ' + phrase);
          }
					out += (audio[phrase] || phrase.indexOf('$') >= 0) ? sayAsAudio(phrase, audio[phrase], element) : sayAsTTS(phrase, element);
					phrase = '';
					audioStream = false;
				}
			}
		}
		out += sayAsTTS(phrase, element);
		return out;
	};

	var fields = [];
	_.each(node.blocks, function(block) {
    var dtmf, voice, rule;
		if (block.type === 'form') {
			print('\n<form>');

          print('\n\t<var name="gNoInputEventCount" expr="0" />');
      		print('\n\t<var name="gNoMatchEventCount" expr="0" />');
			_.each(block.elements, function(element) {
				if (element.type === 'txt') {
					print('\n\t<block>');
					print(speak(element, element.txt, flow.raw.audio));
					if (node.continueNode) {
						print('\n\t</block>');
						print('\n\t<field name="finishedHearing">');
            print('<property name="timeout" value="0.1s" />');
            print('\n\t\t<grammar xml:lang="en-us" version="1.0" root="finishedRule" mode="dtmf"><rule id="finishedRule" scope="public"><item>@</item></rule></grammar>');
            print('\n\t\t<nomatch />');
						print('\n\t\t<noinput count="1"><goto nextitem="submit" /></noinput>');
						print('\n\t</field>');
						print('\n\t<block name="submit">');
						print('\n\t\t<goto next="${internal.nextURL}?_dt=$DateUtil.now().getTime()" />');
					}
					print('\n\t</block>');
				} else if (element.type === 'field') {

					print('\n\t<field ');
					if (element.name) {
						fields.push(element.name);
						print('name="' + element.name + '" ');
					}
          // use a type to over-ride only if grammars are not present
          if ((element.dtmf && !element.dtmfGrammar) || (element.voice && !element.voiceGrammar)) {
            if (element.fieldType) {
              print('type="' + element.fieldType);
            }
            if (element.fieldMeta) {
              if (element.fieldMeta.indexOf('..') === -1) {
                print('?length=' + element.fieldMeta);
              } else {
                var range = element.fieldMeta.split('..');
                if (range.length === 2 && element.fieldMeta.startsWith('..')) {
                  print('?maxlength=' + range[1]);
                } else if (range.length === 2 && element.fieldMeta.endsWith('..')) {
                  print('?minlength=' + range[0]);
                } else if (range.length === 2) {
                  print('?minlength=' + range[0] + ';maxlength=' + range[1]);
                }
              }
            }
            if (element.fieldType) {
              print('"');
            }
          }
					print('>');
					if (element.dtmf && ! element.voice) {
						print('\n\t\t<property name="inputmodes" value="dtmf" />');
					} else if (! element.dtmf && element.voice) {
						print('\n\t\t<property name="inputmodes" value="voice" />');
					}
					rule = element.name + 'Rule';

          if (element.name && element.dtmf && element.dtmfGrammar) {
            dtmf = element.dtmfGrammar.split(/\s*,\s*/);
            if (dtmf.length) {
              print('\n\t\t<grammar type="application/grammar-xml" xml:lang="en-US" version="1.0" root="' + rule + '" mode="dtmf">');
              print('\n\t\t\t<rule id="' + rule + '" scope="public">');
              print('\n\t\t\t<one-of>');
              for (var di = 0; di < dtmf.length; di++) {
                if (dtmf[di]) {
                	print('\n\t\t\t\t<item>' + dtmf[di] + '</item>');
                }
              }
              print('\n\t\t\t</one-of>');
              print('\n\t\t\t</rule>');
              print('\n\t\t</grammar>');
            }
          }
          if (element.name && element.voice && element.voiceGrammar) {
            voice = element.voiceGrammar.split(/\s*,\s*/);
            if (voice.length) {
              print('\n\t\t<grammar type="application/grammar-xml" xml:lang="en-US" version="1.0" root="' + rule + '" mode="voice">');
              print('\n\t\t\t<rule id="' + rule + '" scope="public">');
              print('\n\t\t\t<one-of>');
              for (var di = 0; di < voice.length; di++) {
                if (voice[di]) {
                	print('\n\t\t\t\t<item>' + voice[di] + '</item>');
                }
              }
              print('\n\t\t\t</one-of>');
              print('\n\t\t\t</rule>');
              print('\n\t\t</grammar>');
            }
          }

					if (element.txt) {
						print(speak(element, element.txt, flow.raw.audio));
					}

          // exit if reprompt times and goodBye message is set
          if (element.reprompt) {
           	print('\n\t\t<noinput count="'+ element.reprompt +'">');
            if (element.goodBye) {
            	print(speak(element, element.goodBye, flow.raw.audio));
            }
            print('\n\t\t<var name="gNoInputEventCount" expr="gNoInputEventCount+1" />');
            print('\n\t\t<submit namelist="gNoInputEventCount gNoMatchEventCount" next="${internal.nextURL}?_dt=$DateUtil.now().getTime()" />');
            print('\n\t\t</noinput>');
						print('\n\t\t<nomatch count="'+ element.reprompt +'">');
            if (element.goodBye) {
            	print(speak(element, element.goodBye, flow.raw.audio));
            }
            print('\n\t\t<var name="gNoMatchEventCount" expr="gNoMatchEventCount+1" />');
            print('\n\t\t<submit namelist="gNoInputEventCount gNoMatchEventCount" next="${internal.nextURL}?_dt=$DateUtil.now().getTime()" />');
            print('\n\t\t</nomatch>');
          }

					print('\n\t\t<noinput>');
          if (element.noInput) {
						print(speak(element, element.noInput, flow.raw.audio));
          }
          print('<reprompt />');
					print('\n\t\t</noinput>');

					print('\n\t\t<nomatch>');
          if (element.noMatch) {
						print(speak(element, element.noMatch, flow.raw.audio));
          }
          print('<reprompt />');
					print('\n\t\t</nomatch>');

					print('\n\t</field>');
				}
			})
			if (fields.length) {
				print('\n\t<filled mode="all">');
				print('\n\t\t<submit namelist="' + fields.join(' ') + '" next="${internal.nextURL}?_dt=$DateUtil.now().getTime()" />');
				print('\n\t</filled>');
			}
			print('\n</form>');
		}
	});

%>
</vxml>